In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
from category_encoders import BinaryEncoder


In [157]:
df= pd.read_csv("used_cars.csv")
df.head()


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [158]:
len(df["model"].unique())


1898

In [159]:
def model(df):
    df['model'] = df.groupby('brand').cumcount() + 1

model(df)




In [160]:



from category_encoders import BinaryEncoder
encoder = BinaryEncoder(cols=['brand'])
df = encoder.fit_transform(df)



In [161]:
def mileage(df):
    for index, row in df.iterrows():
        df.at[index, 'milage'] = row['milage'][0:-3]

mileage(df)


In [162]:
df['fuel_type'].unique()



array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [163]:
fuel_type_dict = {
    'E85 Flex Fuel':4 ,
    'Gasoline': 1,
    'Hybrid': 3,
    'Diesel': 2,
    'Plug-In Hybrid':5,
    '–': 0,
    'not supported':0 
}

df['fuel_type'] = df['fuel_type'].map(fuel_type_dict)


In [164]:
def horse_power(df):
    
    for index, x in df['engine'].items():
        if "HP" in str(x):
            df.at[index, 'engin_hp'] = str(x).split(".")[0]  
        else:
            df.at[index, 'engin_hp'] = 0  

horse_power(df)


In [165]:
def engine_displacement(df):
    for index, x in df['engine'].items():
        if 'L' in str(x):
            value = (x).split('L')[0][-3:]  
            df.at[index, 'engine_displacement'] = value
        elif "Liter" in str(x):
            value = str(x).split('Liter')[0][-4:]
            df.at[index, 'engine_displacement'] = value
        else:
            pass   

engine_displacement(df)





In [166]:
import re

def engine_displacement(df):
    for index, x in df['engine'].items():
        x = str(x)
        
        
        match = re.search(r'(\d+(\.\d+)?)\s*(L|Liter)', x)
        
        if match:
            displacement = match.group(1)  
            df.at[index, 'engine_displacement'] = displacement



engine_displacement(df)




In [167]:
df['transmission'].unique()


array(['6-Speed A/T', '8-Speed Automatic', 'Automatic', '7-Speed A/T',
       'F', 'A/T', '8-Speed A/T', 'Transmission w/Dual Shift Mode',
       '9-Speed Automatic', '6-Speed M/T', '10-Speed A/T', '9-Speed A/T',
       '5-Speed A/T', '1-Speed A/T', 'Automatic CVT',
       '7-Speed Automatic with Auto-Shift', 'CVT-F', 'M/T',
       '6-Speed Automatic with Auto-Shift', '10-Speed Automatic',
       'CVT Transmission', '4-Speed A/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T',
       '8-Speed Automatic with Auto-Shift', '5-Speed Automatic',
       '8-SPEED AT', '1-Speed Automatic', '5-Speed M/T', 'Manual',
       '6-Speed Manual', 'Variable', '2', '7-Speed Manual',
       'Automatic, 9-Spd 9G-Tronic', 'Automatic, 8-Spd',
       'Automatic, 8-Spd Sport w/Sport & Manual Modes', '–',
       'Auto, 6-Spd w/CmdShft', 'Transmission Overdrive Switch',
       '7-Speed Automatic', 'Automatic, 8-Spd PDK Dual-Clutch',
       'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Ma

In [168]:
import pandas as pd

def extract_transmission_features(df):
    transmission_type = []
    num_gears = []
    
    for x in df['transmission']:
        
        if 'Automatic' in str(x):
            transmission_type.append('Automatic')
        elif 'Manual' in str(x):
            transmission_type.append('Manual')
        else:
            transmission_type.append('Other')
        
      
        digits = [char for char in str(x) if char.isdigit()]
        if digits:
            num_gears.append(int(''.join(digits)))
        else:
            num_gears.append(None)
    
    df['transmission_type'] = transmission_type
    df['num_gears'] = num_gears



extract_transmission_features(df)
df.drop(columns=['transmission','clean_title'], inplace=True)


In [169]:
df['accident'].unique()


array(['At least 1 accident or damage reported', 'None reported', nan],
      dtype=object)

In [170]:
df['accident'] = df['accident'].apply(lambda x: 1 if x == 'At least 1 accident or damage reported' else 0)





In [171]:
df['transmission_type'].value_counts()


transmission_type
Other        3279
Automatic     715
Manual         15
Name: count, dtype: int64

In [172]:
labels = {'Other': 0, 'Automatic': 1, 'Manual': 2}
df['transmission_type'] = df['transmission_type'].map(labels)

df.head()


,brand_0,brand_1,brand_2,brand_3,brand_4,brand_5,model,model_year,milage,fuel_type,engine,ext_col,int_col,accident,price,engin_hp,engine_displacement,transmission_type,num_gears
0,0,0,0,0,0,1,1,2013,"51,000",4.0,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,Black,Black,1,"$10,300",300,3.7,0,6.0
1,0,0,0,0,1,0,1,2021,"34,742",1.0,3.8L V6 24V GDI DOHC,Moonlight Cloud,Gray,1,"$38,005",0,3.8,1,8.0
2,0,0,0,0,1,1,1,2022,"22,372",1.0,3.5 Liter DOHC,Blue,Black,0,"$54,598",0,3.5,1,NaN
3,0,0,0,1,0,0,1,2015,"88,900",3.0,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,Black,Black,0,"$15,500",354,3.5,0,7.0
4,0,0,0,1,0,1,1,2021,"9,835",1.0,2.0L I4 16V GDI DOHC Turbo,Glacier White Metallic,Black,0,"$34,999",0,2.0,1,8.0


In [173]:
df['ext_col'].value_counts()


ext_col
Black                       905
White                       816
Gray                        496
Silver                      374
Blue                        349
                           ... 
Anthracite Blue Metallic      1
Moonlight Cloud               1
Silver Mist                   1
Cirrus Silver Metallic        1
C / C                         1
Name: count, Length: 319, dtype: int64

In [174]:
df['int_col'].value_counts()


int_col
Black               2025
Beige                538
Gray                 472
Brown                157
–                    133
                    ... 
Almond Beige           1
Kyalami Orange         1
Sakhir Orange          1
Tan/Ebony              1
Black / Graphite       1
Name: count, Length: 156, dtype: int64

In [175]:
ext_list = {'Black': 0, 'White': 1, 'Gray': 2, 'Silver': 3}

def ext_col(df):
    df['ext_col'] = df['ext_col'].map(ext_list).fillna(-1)
    return df


df = ext_col(df)




In [176]:
df['price'] = df['price'].apply(lambda x: x[1:])
df['price'] = df['price'].apply(lambda x: x.replace(',',''))
df['milage']=df['milage'].apply(lambda x: x.replace(',',''))


In [177]:
frequency_mapping = {'Black':4 , 'Beige':1 , 'Gray': 2, 'Brown':3 , '–': 0}
df['int_col'] = df['int_col'].map(frequency_mapping).fillna(-1)




In [178]:
df.head()


,brand_0,brand_1,brand_2,brand_3,brand_4,brand_5,model,model_year,milage,fuel_type,engine,ext_col,int_col,accident,price,engin_hp,engine_displacement,transmission_type,num_gears
0,0,0,0,0,0,1,1,2013,51000,4.0,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,0.0,4.0,1,10300,300,3.7,0,6.0
1,0,0,0,0,1,0,1,2021,34742,1.0,3.8L V6 24V GDI DOHC,-1.0,2.0,1,38005,0,3.8,1,8.0
2,0,0,0,0,1,1,1,2022,22372,1.0,3.5 Liter DOHC,-1.0,4.0,0,54598,0,3.5,1,NaN
3,0,0,0,1,0,0,1,2015,88900,3.0,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,0.0,4.0,0,15500,354,3.5,0,7.0
4,0,0,0,1,0,1,1,2021,9835,1.0,2.0L I4 16V GDI DOHC Turbo,-1.0,4.0,0,34999,0,2.0,1,8.0


In [179]:
df.drop(columns=['engine'],inplace=True)


In [180]:
df = df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)


In [181]:
df.head()


,brand_0,brand_1,brand_2,brand_3,brand_4,brand_5,model,model_year,milage,fuel_type,ext_col,int_col,accident,price,engin_hp,engine_displacement,transmission_type,num_gears
0,0,0,0,0,0,1,1,2013,51000,4,0,4,1,10300,300,3,0,6
1,0,0,0,0,1,0,1,2021,34742,1,-1,2,1,38005,0,3,1,8
2,0,0,0,0,1,1,1,2022,22372,1,-1,4,0,54598,0,3,1,0
3,0,0,0,1,0,0,1,2015,88900,3,0,4,0,15500,354,3,0,7
4,0,0,0,1,0,1,1,2021,9835,1,-1,4,0,34999,0,2,1,8


In [182]:
df.isna().sum()


brand_0                0
brand_1                0
brand_2                0
brand_3                0
brand_4                0
brand_5                0
model                  0
model_year             0
milage                 0
fuel_type              0
ext_col                0
int_col                0
accident               0
price                  0
engin_hp               0
engine_displacement    0
transmission_type      0
num_gears              0
dtype: int64

In [183]:
df.isna().sum()


brand_0                0
brand_1                0
brand_2                0
brand_3                0
brand_4                0
brand_5                0
model                  0
model_year             0
milage                 0
fuel_type              0
ext_col                0
int_col                0
accident               0
price                  0
engin_hp               0
engine_displacement    0
transmission_type      0
num_gears              0
dtype: int64

In [184]:
df.fillna(0, inplace=True)


In [185]:

mean_hp = df['engin_hp'][df['engin_hp'] != 0].mean()
df['engin_hp'] = df['engin_hp'].replace(0, mean_hp)

mode_displacement = df['engine_displacement'][df['engine_displacement'] != 0].mode()[0]
df['engine_displacement'] = df['engine_displacement'].replace(0, mode_displacement)

mode_gears = df['num_gears'][df['num_gears'] != 0].mode()[0]
df['num_gears'] = df['num_gears'].replace(0, mode_gears)


In [ ]:
df.head(30)


,brand_0,brand_1,brand_2,brand_3,brand_4,brand_5,model,model_year,milage,fuel_type,ext_col,int_col,accident,price,engin_hp,engine_displacement,transmission_type,num_gears
0,0,0,0,0,0,1,1,2013,51000,4,0,4,1,10300,300.000000,3,0,6
1,0,0,0,0,1,0,1,2021,34742,1,-1,2,1,38005,332.283839,3,1,8
2,0,0,0,0,1,1,1,2022,22372,1,-1,4,0,54598,332.283839,3,1,6
3,0,0,0,1,0,0,1,2015,88900,3,0,4,0,15500,354.000000,3,0,7
4,0,0,0,1,0,1,1,2021,9835,1,-1,4,0,34999,332.283839,2,1,8
5,0,0,0,1,1,0,1,2016,136397,1,3,-1,0,14798,332.283839,2,0,6
6,0,0,0,1,0,1,2,2017,84000,1,-1,4,0,31000,292.000000,2,0,6
7,0,0,0,1,1,1,1,2001,242000,1,-1,-1,0,7300,282.000000,4,0,6
8,0,0,0,0,1,1,2,2021,23436,1,0,4,0,41927,311.000000,3,0,6
9,0,0,1,0,0,0,1,2020,34000,0,0,4,0,69950,534.000000,3,0,6


In [ ]:
# df.to_csv('cleaned_car_data.csv', index=False)
